# 論文 6：指標網路（Pointer Networks）
## Oriol Vinyals, Meire Fortunato, Navdeep Jaitly

### 實作：基於注意力的指向機制

指標網路使用注意力來指向輸入元素，解決組合問題如凸包和旅行推銷員問題（TSP）。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull

np.random.seed(42)

## 用於指向的注意力機制

In [ ]:
def softmax(x, axis=-1):
    """穩定的 softmax"""
    x_max = np.max(x, axis=axis, keepdims=True)
    exp_x = np.exp(x - x_max)
    return exp_x / np.sum(exp_x, axis=axis, keepdims=True)

class PointerAttention:
    def __init__(self, hidden_size):
        self.hidden_size = hidden_size
        
        # 注意力參數
        self.W1 = np.random.randn(hidden_size, hidden_size) * 0.1
        self.W2 = np.random.randn(hidden_size, hidden_size) * 0.1
        self.v = np.random.randn(hidden_size, 1) * 0.1
    
    def forward(self, encoder_states, decoder_state):
        """
        計算對輸入元素的注意力分數
        
        encoder_states: (seq_len, hidden_size) - 編碼的輸入
        decoder_state: (hidden_size, 1) - 當前解碼器狀態
        
        回傳：
        probs: (seq_len, 1) - 對輸入的指標分佈
        """
        seq_len = encoder_states.shape[0]
        
        # 計算注意力分數
        scores = []
        for i in range(seq_len):
            # e_i = v^T * tanh(W1*encoder_state + W2*decoder_state)
            encoder_proj = np.dot(self.W1, encoder_states[i:i+1].T)
            decoder_proj = np.dot(self.W2, decoder_state)
            score = np.dot(self.v.T, np.tanh(encoder_proj + decoder_proj))
            scores.append(score[0, 0])
        
        scores = np.array(scores).reshape(-1, 1)
        
        # Softmax 得到機率
        probs = softmax(scores, axis=0)
        
        return probs, scores

# 測試注意力
hidden_size = 32
attention = PointerAttention(hidden_size)

# 虛擬的編碼器狀態和解碼器狀態
seq_len = 5
encoder_states = np.random.randn(seq_len, hidden_size)
decoder_state = np.random.randn(hidden_size, 1)

probs, scores = attention.forward(encoder_states, decoder_state)
print(f"指標網路注意力已初始化")
print(f"注意力機率總和：{probs.sum():.4f}")
print(f"機率形狀：{probs.shape}")

## 完整的指標網路架構

In [ ]:
class PointerNetwork:
    def __init__(self, input_size, hidden_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # 編碼器（簡單 RNN）
        self.encoder_Wx = np.random.randn(hidden_size, input_size) * 0.1
        self.encoder_Wh = np.random.randn(hidden_size, hidden_size) * 0.1
        self.encoder_b = np.zeros((hidden_size, 1))
        
        # 解碼器（RNN）
        self.decoder_Wx = np.random.randn(hidden_size, input_size) * 0.1
        self.decoder_Wh = np.random.randn(hidden_size, hidden_size) * 0.1
        self.decoder_b = np.zeros((hidden_size, 1))
        
        # 指標機制
        self.attention = PointerAttention(hidden_size)
    
    def encode(self, inputs):
        """
        編碼輸入序列
        inputs: (input_size, 1) 向量的列表
        """
        h = np.zeros((self.hidden_size, 1))
        encoder_states = []
        
        for x in inputs:
            h = np.tanh(
                np.dot(self.encoder_Wx, x) + 
                np.dot(self.encoder_Wh, h) + 
                self.encoder_b
            )
            encoder_states.append(h.flatten())
        
        return np.array(encoder_states), h
    
    def decode_step(self, x, h, encoder_states):
        """
        單一解碼步驟
        """
        # 更新解碼器隱藏狀態
        h = np.tanh(
            np.dot(self.decoder_Wx, x) + 
            np.dot(self.decoder_Wh, h) + 
            self.decoder_b
        )
        
        # 計算指標分佈
        probs, scores = self.attention.forward(encoder_states, h)
        
        return probs, h, scores
    
    def forward(self, inputs, targets=None):
        """
        完整前向傳遞
        """
        # 編碼輸入
        encoder_states, h = self.encode(inputs)
        
        # 解碼（指向輸入）
        output_probs = []
        output_indices = []
        
        # 起始標記（使用輸入的平均值）
        x = np.mean([inp for inp in inputs], axis=0)
        
        for step in range(len(inputs)):
            probs, h, scores = self.decode_step(x, h, encoder_states)
            output_probs.append(probs)
            
            # 取樣指標
            ptr_idx = np.argmax(probs)
            output_indices.append(ptr_idx)
            
            # 下一個輸入是被指向的元素
            x = inputs[ptr_idx]
        
        return output_indices, output_probs

print("指標網路架構已建立")

## 任務：凸包問題

給定一組 2D 點，按凸包順序輸出它們

In [ ]:
def generate_convex_hull_data(num_samples=20, num_points=10):
    """
    生成隨機 2D 點及其凸包順序
    """
    data = []
    
    for _ in range(num_samples):
        # 生成隨機點
        points = np.random.rand(num_points, 2)
        
        # 計算凸包
        try:
            hull = ConvexHull(points)
            hull_indices = hull.vertices.tolist()
            
            # 將點轉換為輸入格式
            inputs = [points[i:i+1].T for i in range(num_points)]
            
            data.append({
                'points': points,
                'inputs': inputs,
                'hull_indices': hull_indices
            })
        except:
            # 跳過退化情況
            continue
    
    return data

# 生成資料
convex_hull_data = generate_convex_hull_data(num_samples=10, num_points=8)
print(f"已生成 {len(convex_hull_data)} 個凸包範例")

# 視覺化範例
example = convex_hull_data[0]
points = example['points']
hull_indices = example['hull_indices']

plt.figure(figsize=(8, 8))
plt.scatter(points[:, 0], points[:, 1], s=100, alpha=0.6)

# 繪製凸包
for i in range(len(hull_indices)):
    start = hull_indices[i]
    end = hull_indices[(i + 1) % len(hull_indices)]
    plt.plot([points[start, 0], points[end, 0]], 
             [points[start, 1], points[end, 1]], 
             'r-', linewidth=2)

# 標記點
for i, (x, y) in enumerate(points):
    plt.text(x, y, str(i), fontsize=12, ha='center', va='center')

plt.title('凸包任務')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.show()

print(f"\n凸包順序：{hull_indices}")

## 在凸包上測試指標網路

In [ ]:
# 建立指標網路
ptr_net = PointerNetwork(input_size=2, hidden_size=32)

# 在範例上測試
test_example = convex_hull_data[0]
inputs = test_example['inputs']
true_hull = test_example['hull_indices']

# 前向傳遞（未訓練）
predicted_indices, probs = ptr_net.forward(inputs)

print("未訓練的指標網路：")
print(f"真實凸包順序：{true_hull}")
print(f"預測順序：{predicted_indices}")

# 視覺化每一步的注意力
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for step in range(min(8, len(probs))):
    ax = axes[step]
    
    # 繪製點
    ax.scatter(points[:, 0], points[:, 1], s=200, alpha=0.3, c='gray')
    
    # 突顯注意力權重
    attention_weights = probs[step].flatten()
    for i, (x, y) in enumerate(points):
        ax.scatter(x, y, s=1000*attention_weights[i], alpha=0.6, c='red')
        ax.text(x, y, str(i), fontsize=10, ha='center', va='center')
    
    ax.set_title(f'步驟 {step}：指向 {predicted_indices[step]}')
    ax.set_xlim(-0.1, 1.1)
    ax.set_ylim(-0.1, 1.1)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('指標網路注意力（未訓練）', y=1.02, fontsize=14)
plt.show()

## 更簡單的任務：數字排序

一個更簡單的展示，網路學習排序。

In [ ]:
def generate_sorting_data(num_samples=50, seq_len=5):
    """
    生成隨機序列及其排序順序
    """
    data = []
    
    for _ in range(num_samples):
        # 隨機值
        values = np.random.rand(seq_len)
        
        # 排序索引
        sorted_indices = np.argsort(values).tolist()
        
        # 轉換為輸入格式（1D 值）
        inputs = [np.array([[v]]) for v in values]
        
        data.append({
            'values': values,
            'inputs': inputs,
            'sorted_indices': sorted_indices
        })
    
    return data

# 生成排序資料
sort_data = generate_sorting_data(num_samples=20, seq_len=6)

# 測試範例
example = sort_data[0]
print("排序任務範例：")
print(f"值：{example['values']}")
print(f"排序順序（索引）：{example['sorted_indices']}")
print(f"排序後的值：{example['values'][example['sorted_indices']]}")

# 視覺化
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.bar(range(len(example['values'])), example['values'])
plt.title('原始順序')
plt.xlabel('索引')
plt.ylabel('值')

plt.subplot(1, 2, 2)
sorted_vals = example['values'][example['sorted_indices']]
plt.bar(range(len(sorted_vals)), sorted_vals)
plt.title('排序順序')
plt.xlabel('排序序列中的位置')
plt.ylabel('值')

plt.tight_layout()
plt.show()

## 關鍵要點

### 指標網路創新：
1. **輸出詞彙表就是輸入**：網路指向輸入元素
2. **可變輸出大小**：可以處理不同的輸入長度
3. **無固定詞彙表**：解決組合問題
4. **注意力作為選擇**：使用注意力機制來「指向」

### 應用：
- 凸包計算
- 旅行推銷員問題（TSP）
- 排序
- Delaunay 三角剖分
- 任何輸出是輸入的排列/子集的問題

### 架構組件：
1. **編碼器**：處理輸入序列
2. **解碼器**：生成指標序列
3. **注意力**：計算對輸入位置的分佈
4. **指向**：選擇下一個要輸出的輸入元素

### 訓練：
- 使用正確指標序列進行監督學習
- 對指標分佈使用交叉熵損失
- 可以使用強化學習進行最佳化問題